In [ ]:
from google.colab import drive
drive.mount('/content/drive')
datadir = "/content/drive/My Drive/"

Mounted at /content/drive


In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
directory = "/content/drive/MyDrive/cs545/FinalProject/"

Mounted at /content/drive


In [16]:
!pip install pydub

In [17]:
import numpy as np
import librosa
import librosa.display
from pydub import AudioSegment
from IPython.display import Audio

def wiener_filter_audio(noisy_audio, noise_power, signal_power):
    # Apply the Wiener filter to the audio
    denoised_audio = (signal_power / (signal_power + noise_power)) * noisy_audio

    return denoised_audio

# Load a noisy audio file for demonstration
file_path = directory + 'noisy_input.wav'  # Update with your file path
noisy_audio, sr = librosa.load(file_path, sr=44100)

# Estimate power spectral densities (for demonstration purposes)
noise_power = np.mean(np.abs(librosa.stft(noisy_audio))**2)
# For simplicity, assume the signal power is the same as the noise power
signal_power = noise_power

# Apply the Wiener filter to the audio
denoised_audio = wiener_filter_audio(noisy_audio, noise_power, signal_power)

# Convert the NumPy array to an AudioSegment
denoised_audio_segment = AudioSegment(
    data=denoised_audio.tobytes(),
    sample_width=denoised_audio.dtype.itemsize,
    frame_rate=sr,
    channels=1  # Adjust for stereo audio
)

# Display the denoised audio using IPython.display.Audio
Audio(denoised_audio_segment.raw_data, rate=sr)


In [ ]:
!pip install webrtcvad

In [ ]:
import numpy as np
import scipy.signal
import webrtcvad

def apply_vad(audio, sample_rate=16000, vad_mode=3):
    vad = webrtcvad.Vad(vad_mode)
    frame_duration = 30  # ms
    frame_size = int(sample_rate * frame_duration / 1000)

    segments = []
    for start in np.arange(0, len(audio), frame_size):
        end = min(start + frame_size, len(audio))
        frame = audio[start:end]
        if len(frame) > 0 and vad.is_speech(frame.tobytes(), sample_rate):
            segments.append(frame)

    return np.concatenate(segments)


def apply_wiener_filter(noisy_signal, noise_psd, signal_psd):
    alpha = 0.99  # Smoothing factor
    noise_estimation = alpha * noise_psd + (1 - alpha) * np.abs(noisy_signal)**2
    wiener_filter = noise_estimation / (noise_estimation + signal_psd)
    return np.real(scipy.signal.fftconvolve(noisy_signal, wiener_filter, 'same'))

# Example usage
if __name__ == "__main__":
    # Load your audio data (replace 'your_audio_file.wav' with your actual file)
    sample_rate, audio_data = scipy.io.wavfile.read(datadir + 'vl1.wav')

    # Apply VAD
    vad_result = apply_vad(audio_data, sample_rate=sample_rate)

    # Estimate noise PSD and signal PSD (for simplicity, you may need more sophisticated methods)
    noise_psd = np.abs(np.fft.fft(vad_result))**2
    signal_psd = np.abs(np.fft.fft(audio_data))**2

    # Apply Wiener filter
    denoised_signal = apply_wiener_filter(audio_data, noise_psd, signal_psd)

    # Save the denoised signal to a new audio file
    scipy.io.wavfile.write('denoised_output.wav', sample_rate, denoised_signal)


Error: ignored

In [2]:
# ! pip install webrtcvad

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73462 sha256=995735cbda579aae2decdb30ed888d5e2fce24143dd819ca9a4ff72c019a0f5e
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
Successfully built webrtcvad


In [14]:
import numpy as np
import scipy.io.wavfile
import scipy.signal
import webrtcvad

def apply_vad(audio, sample_rate=16000, vad_mode=3):
    vad = webrtcvad.Vad(vad_mode)
    frame_duration = 30  # ms
    frame_size = int(sample_rate * frame_duration / 1000)
    # print("frame_size :", frame_size)
    segments = []
    for start in np.arange(0, len(audio), frame_size):
        end = min(start + frame_size, len(audio))
        frame = audio[start:end]
        # print(len(frame))
        if len(frame) >= frame_size and vad.is_speech(frame.tobytes(), sample_rate):
            segments.append(frame)

    return np.concatenate(segments)

def apply_wiener_filter(noisy_signal, noise_psd, signal_psd):
    alpha = 0.99  # Smoothing factor
    noise_estimation = alpha * noise_psd + (1 - alpha) * np.abs(noisy_signal)**2
    wiener_filter = noise_estimation / (noise_estimation + signal_psd)
    return np.real(scipy.signal.fftconvolve(noisy_signal, wiener_filter, 'same'))

def calculate_mse(true_signal, estimated_signal):
    return np.mean((true_signal - estimated_signal)**2)

# Load your audio data (replace 'your_audio_file.wav' with your actual file)
sample_rate, audio_data = scipy.io.wavfile.read(directory + 'noisy_input.wav')
print("audio_data shape : ", audio_data.shape)


audio_data = audio_data[:-48]
print("audio_data shape : ", audio_data.shape)

# Assuming you have the true signal (replace 'true_signal_file.wav' with your actual file)
true_sample_rate, true_signal = scipy.io.wavfile.read(directory + 'clean_input.wav')

# Apply VAD
vad_result = apply_vad(audio_data, sample_rate=sample_rate)

# Estimate noise PSD and signal PSD (for simplicity, you may need more sophisticated methods)
noise_psd = np.abs(np.fft.fft(vad_result))**2
signal_psd = np.abs(np.fft.fft(audio_data))**2

print("vad_result shape : ", vad_result.shape)
print("noise_psd shape : ", noise_psd.shape)
print("signal_psd shape : ", signal_psd.shape)
print("audio_data shape : ", audio_data.shape)

# Apply Wiener filter
denoised_signal = apply_wiener_filter(audio_data, noise_psd, signal_psd)

# Calculate the MSE
mse = calculate_mse(true_signal[:len(denoised_signal)], denoised_signal)

print(f"MSE: {mse}")

# Save the denoised signal to a new audio file
scipy.io.wavfile.write(directory + 'denoised_output.wav', sample_rate, denoised_signal)

# Print or visualize the true and estimated signals if needed
print("True Signal:", true_signal[:len(denoised_signal)])
print("Estimated Signal:", denoised_signal)


audio_data shape :  (52608,)
audio_data shape :  (52560,)
vad_result shape :  (52560,)
noise_psd shape :  (52560,)
signal_psd shape :  (52560,)
audio_data shape :  (52560,)
MSE: 80465592554.3835
True Signal: [0 0 0 ... 1 1 1]
Estimated Signal: [-285852.55582099 -285382.83486218 -284831.81063958 ...  -43197.2599922
  -42671.07077476  -42974.37872211]
